<H1><center>Segmentation and Clustering Assignment</center></H1>

Installed BeautifulSoup4

In [8]:
pip install BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.


<h2>Imported all the required libraries</h2>

In [6]:
import requests as re
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

<h2>Got data from wikipedia and extracted the content</h2>

In [9]:
result = re.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = bs(result.content, 'html.parser')

<h2>Extracted text array from table</h2>

In [14]:
rows = soup.find('tbody').select('tr')
textArray = [r.get_text() for r in rows]

<h2>Data refining</h2>

In [19]:
df = pd.DataFrame(textArray)
df = df[0].str.split('\n', expand=True)
df = df.rename(columns=df.iloc[0])
df = df.drop(df.index[0])
df = df[df.Borough != 'Not assigned']
df = df.groupby(['Postcode', 'Borough'], sort = False).agg(','.join)
df.reset_index(inplace = True)
df = df.replace("Not assigned", "Queen's Park")
df.shape

(103, 3)